In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

In [2]:
#Store Part I results (from WeatherPy) into DataFrame and Load the csv exported in Part I to a DataFrame.

weather_csv_file = "../WeatherPy/Output/weather_df.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,Unnamed: 0,City name,Country,Cloudiness,Dates,Humidities,Lats,Lngs,Max_temps,Wind_speeds
0,0,esil,KZ,0,1594849231,66,51.96,66.40,68.90,8.21
1,1,hilo,US,90,1594849772,65,19.73,-155.09,82.40,9.17
2,2,ponta delgada,PT,100,1594849297,90,38.73,-28.27,65.68,8.52
3,3,puerto madryn,ES,0,1594849297,42,36.59,-6.23,82.40,23.04
4,4,punta arenas,PH,99,1594849772,87,8.99,125.34,75.18,3.83



Out[2]:
City	Cloudiness	Country	Date	Humidity	Latitude	Longitude	Max Temp	Wind Speed
0	hasaki	75.0	JP	1.580018e+09	86.0	35.73	140.83	48.00	5.82
1	oktyabrskoye	0.0	RU	1.580018e+09	85.0	43.06	44.74	19.00	6.71
2	atuona	37.0	PF	1.580018e+09	76.0	-9.80	-139.03	81.00	14.47
3	rikitea	24.0	PF	1.580018e+09	78.0	-23.12	-134.97	78.73	11.10
4	bredasdorp	69.0	ZA	1.580018e+09	77.0	-34.53	20.04	63.00	12.75

In [3]:

# Humidity Heatmap
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.



# Configure gmaps with API key.
gmaps.configure(api_key=gkey)

# Also, handle NaN values
weather_df = weather_df.dropna()

# Convert Humidity to float and store
humidity = weather_df["Humidities"].astype(float)
humidity

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lats", "Lngs"]].astype(float)
locations
# Create a humidity Heatmap layer
figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(layout=figure_layout)
fig
# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidities"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='1000px'))

In [4]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max_temps"] < 80) & (filtered_weather_df["Max_temps"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind_speeds"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,Unnamed: 0,City name,Country,Cloudiness,Dates,Humidities,Lats,Lngs,Max_temps,Wind_speeds
24,24,bismil,TR,0,1594849774,53,37.85,40.67,75.20,4.70
96,96,rio muerto,GR,0,1594849303,78,38.30,21.78,73.40,5.41
158,158,aljezur,PT,0,1594849245,66,37.32,-8.80,70.23,3.40
181,181,estacion coahuila,PH,0,1594849310,90,15.70,120.61,77.22,3.15
201,201,pilar,BR,0,1594849790,78,-9.60,-35.96,77.00,6.93
276,276,salym,RU,0,1594849799,76,60.06,71.48,72.54,7.58
313,313,rio grande,GR,0,1594849802,78,38.30,21.78,73.40,5.41
396,396,santa isabel do rio negro,PH,0,1594849810,83,17.49,120.43,77.97,4.32
435,435,artyom,AZ,0,1594849273,57,40.47,50.33,78.80,9.17
448,448,qostanay,KZ,0,1594849274,49,53.17,63.58,73.40,4.47


In [5]:
hotel_df =filtered_weather_df

hotel_df['Hotel Name'] = np.nan
hotel_df
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

idx = 0
for index, row in hotel_df.iterrows():
    # set up a parameters dictionary
    params = {
        "location": f"{row['Lats']}, {row['Lngs']}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
        
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    if len(results) > 0:
        name = response_json['results'][0]['name']
        hotel_df.iloc[idx, -1] = name
    idx = idx + 1

narrowed_city_df = hotel_df  
narrowed_city_df.dropna()


,Unnamed: 0,City name,Country,Cloudiness,Dates,Humidities,Lats,Lngs,Max_temps,Wind_speeds,Hotel Name
24,24,bismil,TR,0,1594849774,53,37.85,40.67,75.20,4.70,Hotel Dolay
96,96,rio muerto,GR,0,1594849303,78,38.30,21.78,73.40,5.41,PORTO RIO HOTEL & CASINO
158,158,aljezur,PT,0,1594849245,66,37.32,-8.80,70.23,3.40,Hotel Vale Da Telha
181,181,estacion coahuila,PH,0,1594849310,90,15.70,120.61,77.22,3.15,La Romana Countryside Haven
201,201,pilar,BR,0,1594849790,78,-9.60,-35.96,77.00,6.93,Refúgios Motel
276,276,salym,RU,0,1594849799,76,60.06,71.48,72.54,7.58,Gostinitsa V Salyme
313,313,rio grande,GR,0,1594849802,78,38.30,21.78,73.40,5.41,PORTO RIO HOTEL & CASINO
435,435,artyom,AZ,0,1594849273,57,40.47,50.33,78.80,9.17,F marine resort
448,448,qostanay,KZ,0,1594849274,49,53.17,63.58,73.40,4.47,Hotel Fessla
468,468,dasoguz,TM,0,1594849276,34,41.84,59.97,78.80,9.17,Hotel Şabat


In [6]:

# Assign the marker layer to a variable
locations = narrowed_city_df[["Lats", "Lngs"]]
markers = gmaps.marker_layer(locations)


In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='1000px'))